copy from makec2s.ipynb

In [4]:
import numpy as np
import random

# inspired by int ConnectionMatrix::genFlowBytes()
def genflowbytes():
    mean_bytes = 100.0 * 1024
    shape = 1.05
    scale = mean_bytes * (shape - 1)/shape

    x = np.random.exponential(scale=1.0/shape)
    flowbytes = int(scale * np.exp(x))
    return flowbytes

def adjustbytesbymtu(flowbytes):
  mss = 1500
  return mss * ((flowbytes+mss-1)//mss)





swlist = range(120,201,20)
# multlist = [[1,0,0],[1,0,0],[1,0,0],[1,0,0],[1,0,0]]
multlist = [[0,47,100],[0,41,100],[0,36,100],[0,32,100],[0,29,100]]
numport = 96
seed = 0
random.seed(seed)


for inumsw,numsw in enumerate(swlist):
  numserver = numsw*56
  c = int(numsw*0.9)
  # s = int(numsw*0.1)
  s = numsw-c
  # print(f"numsw={numsw},numport={numport},numserver={numserver}")
  mult = multlist[inumsw][0]
  numerator = multlist[inumsw][1]
  denominator = multlist[inumsw][2]
  
  # read leafspine serverfile
  serverfile = f"/home/annzhou/DRing/src/emp/datacentre/serverfiles/dring_{numserver}_{numsw}_{numport}"
  serverdict = dict()
  with open(serverfile,'r') as f:
      lines = f.readlines()
      for line in lines:
          tokens = line.split(',')
          serverdict[int(tokens[0])] = int(tokens[1])

  allfromswitches = list()
  alltoswitches = list()
  while len(allfromswitches)<c:
    rsw = random.randint(0,numsw-1)
    if rsw not in allfromswitches:
      allfromswitches.append(rsw)
  # while len(alltoswitches)<s:
  #   rsw = random.randint(0,numleaf-1)
  #   if rsw not in alltoswitches:
  #     alltoswitches.append(rsw)
  for i in range(numsw):
    if i not in allfromswitches:
      alltoswitches.append(i)

  allfromservers = list()
  alltoservers = list()
  for lsserver in range(numserver):
    lsswitch = serverdict[lsserver]
    if lsswitch in allfromswitches:
      allfromservers.append(lsserver)
    if lsswitch in alltoswitches:
      alltoservers.append(lsserver)

  large_flow_threshold = 10 * 1024 * 1024
  base_flows = list()
  for fromsvr in allfromservers:
    for tosvr in alltoservers:
      if fromsvr!=tosvr:
        mybytes = genflowbytes()
        while mybytes<0 or mybytes>large_flow_threshold:
          mybytes = genflowbytes()
        mybytes = adjustbytesbymtu(mybytes)
        base_flows.append([fromsvr,tosvr,mybytes])

  flowfile = f"/home/annzhou/DRing/src/emp/datacentre/flowfiles/c2s_{numserver}_{numsw}_{numport}_{c}_{s}_{mult}_{numerator}_{denominator}_{seed}"
  with open(flowfile,'w') as f:
    if mult>0:
      for m in range(mult):
        for flow in base_flows:
          f.write(f"{flow[0]},{flow[1]},{flow[2]}\n")

    if denominator>0:
      for flow in base_flows:
        r = random.randint(0,denominator-1) # inclusive
        if r<numerator:
          f.write(f"{flow[0]},{flow[1]},{flow[2]}\n")